In [101]:
import pandas as pd
pd.set_option('display.max_columns', None)

#### JOIN 2023 data

In [2]:
import pandas as pd
import os

# Set the directory path where the files are located
directory_path = './Raw data/'

# Find all the xlsx files in the directory with "LCA_Disclosure_Data_FY" in their filenames
files = [f for f in os.listdir(directory_path) if f.endswith('.xlsx') and 'LCA_Disclosure_Data_FY' in f]

# Initialize an empty list to store the dataframes
dfs = []

# Loop through each file, import the data, and append it to the list of dataframes
for file in files:
    file_path = os.path.join(directory_path, file)
    df = pd.read_excel(file_path, engine='openpyxl')
    dfs.append(df)

# Concatenate the dataframes by row
combined_df = pd.concat(dfs, axis=0, ignore_index=True)

In [10]:
combined_df.head()

,CASE_NUMBER,CASE_STATUS,RECEIVED_DATE,DECISION_DATE,ORIGINAL_CERT_DATE,VISA_CLASS,JOB_TITLE,SOC_CODE,SOC_TITLE,FULL_TIME_POSITION,BEGIN_DATE,END_DATE,TOTAL_WORKER_POSITIONS,NEW_EMPLOYMENT,CONTINUED_EMPLOYMENT,CHANGE_PREVIOUS_EMPLOYMENT,NEW_CONCURRENT_EMPLOYMENT,CHANGE_EMPLOYER,AMENDED_PETITION,EMPLOYER_NAME,TRADE_NAME_DBA,EMPLOYER_ADDRESS1,EMPLOYER_ADDRESS2,EMPLOYER_CITY,EMPLOYER_STATE,EMPLOYER_POSTAL_CODE,EMPLOYER_COUNTRY,EMPLOYER_PROVINCE,EMPLOYER_PHONE,EMPLOYER_PHONE_EXT,NAICS_CODE,EMPLOYER_POC_LAST_NAME,EMPLOYER_POC_FIRST_NAME,EMPLOYER_POC_MIDDLE_NAME,EMPLOYER_POC_JOB_TITLE,EMPLOYER_POC_ADDRESS1,EMPLOYER_POC_ADDRESS2,EMPLOYER_POC_CITY,EMPLOYER_POC_STATE,EMPLOYER_POC_POSTAL_CODE,EMPLOYER_POC_COUNTRY,EMPLOYER_POC_PROVINCE,EMPLOYER_POC_PHONE,EMPLOYER_POC_PHONE_EXT,EMPLOYER_POC_EMAIL,AGENT_REPRESENTING_EMPLOYER,AGENT_ATTORNEY_LAST_NAME,AGENT_ATTORNEY_FIRST_NAME,AGENT_ATTORNEY_MIDDLE_NAME,AGENT_ATTORNEY_ADDRESS1,AGENT_ATTORNEY_ADDRESS2,AGENT_ATTORNEY_CITY,AGENT_ATTORNEY_STATE,AGENT_ATTORNEY_POSTAL_CODE,AGENT_ATTORNEY_COUNTRY,AGENT_ATTORNEY_PROVINCE,AGENT_ATTORNEY_PHONE,AGENT_ATTORNEY_PHONE_EXT,AGENT_ATTORNEY_EMAIL_ADDRESS,LAWFIRM_NAME_BUSINESS_NAME,STATE_OF_HIGHEST_COURT,NAME_OF_HIGHEST_STATE_COURT,WORKSITE_WORKERS,SECONDARY_ENTITY,SECONDARY_ENTITY_BUSINESS_NAME,WORKSITE_ADDRESS1,WORKSITE_ADDRESS2,WORKSITE_CITY,WORKSITE_COUNTY,WORKSITE_STATE,WORKSITE_POSTAL_CODE,WAGE_RATE_OF_PAY_FROM,WAGE_RATE_OF_PAY_TO,WAGE_UNIT_OF_PAY,PREVAILING_WAGE,PW_UNIT_OF_PAY,PW_TRACKING_NUMBER,PW_WAGE_LEVEL,PW_OES_YEAR,PW_OTHER_SOURCE,PW_OTHER_YEAR,PW_SURVEY_PUBLISHER,PW_SURVEY_NAME,TOTAL_WORKSITE_LOCATIONS,AGREE_TO_LC_STATEMENT,H-1B_DEPENDENT,WILLFUL_VIOLATOR,SUPPORT_H1B,STATUTORY_BASIS,APPENDIX_A_ATTACHED,PUBLIC_DISCLOSURE,PREPARER_LAST_NAME,PREPARER_FIRST_NAME,PREPARER_MIDDLE_INITIAL,PREPARER_BUSINESS_NAME,PREPARER_EMAIL,EMPLOYER_POC_ADDRESS_1,EMPLOYER_POC_ADDRESS_2,H1B_DEPENDENT,H_1B_DEPENDENT
0,I-200-19268-393467,Certified,2019-09-25,2019-10-01,NaT,H-1B,"APPLICATION ENGINEER, OMS [15-1199.02]",15-1199,"COMPUTER OCCUPATIONS, ALL OTHER",Y,2019-10-07,2022-10-07,1,1,0,0,0,0,0,"JO-ANN STORES, INC.",JO-ANN FABRIC AND CRAFT STORES,5555 DARROW ROAD,NaN,HUDSON,OH,44236,UNITED STATES OF AMERICA,NaN,3304636787,NaN,451120,KLUSKA,KRYSTA,NaN,"MANAGER, TALENT ACQUISITION",5555 DARROW ROAD,NaN,HUDSON,OH,44236,UNITED STATES OF AMERICA,NaN,3304636787,NaN,KRYSTA.KLUSKA@JOANN.COM,Y,TZU-KAI,TZU-KAI,NaN,333 WEST WACKER DRIVE,15TH FLOOR,CHICAGO,IL,60606,UNITED STATES OF AMERICA,NaN,3.122636e+09,NaN,KLO@FRAGOMEN.COM,"FRAGOMEN, DEL REY, BERNSEN & LOEWY, LLP",IL,SUPREME COURT,NaN,N,NaN,5555 Darrow Road,NaN,Hudson,Summit,OH,44224,100000.00,NaN,Year,95118.0,Year,NaN,IV,2018,OES,NaN,NaN,NaN,NaN,Y,N,N,NaN,NaN,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I-200-19268-638983,Certified,2019-09-25,2019-10-01,NaT,H-1B,BI DEVELOPER II,15-1132,"SOFTWARE DEVELOPERS, APPLICATIONS",Y,2020-01-08,2023-01-07,1,0,1,0,0,0,0,DENKEN SOLUTIONS INC.,NaN,9170 IRVINE CENTER DRIVE,SUITE # 200,IRVINE,CA,92618,UNITED STATES OF AMERICA,NaN,8017923416,NaN,541512,MADDULA,RAJ,NaN,SOLUTION ARCHITECT,9170 IRVINE CENTER DRIVE,SUITE # 200,IRVINE,CA,92618,UNITED STATES OF AMERICA,NaN,8017923416,NaN,IMMIGRATION@DENKENSOLUTIONS.COM,Y,YELENA,YELENA,NaN,6622 SOUTHPOINT DR. S,STE 330,JACKSONVILLE,FL,32216,UNITED STATES OF AMERICA,NaN,9.043890e+09,NaN,USCISNOTICES@SCARBOROUGHLAW.COM,"SCARBOROUGH LAW, L.L.C.",FL,FLORIDA SUPREME COURT,NaN,Y,NAVIHEALTH,210 WESTWOOD PL #400,NaN,BRENTWOOD,WILLIAMSON,TN,37027,38.57,38.57,Hour,39.0,Hour,NaN,II,2019,OES,NaN,NaN,NaN,NaN,Y,Y,N,Y,BOTH,NaN,Disclose Business,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,I-200-19268-177184,Certified,2019-09-25,2019-10-01,NaT,H-1B,QUALITY ENGINEER,17-2141,MECHANICAL ENGINEERS,Y,2019-10-03,2022-10-02,1,0,0,0,0,1,0,"EPITEC, INC.",NaN,24800 DENSO DRIVE,STE. 150,SOUTHFIELD,MI,48033,UNITED STATES OF AMERICA,NaN,2483536800,NaN,541511,WILDMAN,TAYLOR,NaN,IMMIGRATION LEAD,24800 DENSO DRIVE,STE. 150,SOUTHFIELD,MI,48033,UNITED STATES OF AMERICA,NaN,2

#### Check and filter Visa classes in df

In [12]:
list( combined_df['VISA_CLASS'].unique())

['H-1B', 'E-3 Australian', 'H-1B1 Chile', 'H-1B1 Singapore']

In [13]:
combined_df.shape

(2128458, 100)

In [14]:
# Filter the dataframe by VISA_CLASS = 'H-1B'
combined_df = combined_df[combined_df['VISA_CLASS'] == 'H-1B']

In [16]:
# 2128458 - 2074099 = 54359 rows were effected
combined_df.shape

(2074099, 100)

In [18]:
# combined_df.to_csv('h1b_rocord_2020_to_2023.csv', index=False)

### Avoid duplicate cases

In [95]:
print(combined_df.shape)
print(len(list( combined_df['CASE_NUMBER'].unique())))
combined_df = combined_df.drop_duplicates(subset='CASE_NUMBER', keep='last')
print(combined_df.shape)

(2074099, 100)
1749116
(1749116, 100)


### Create database

In [38]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2074099 entries, 0 to 2128457
Data columns (total 100 columns):
 #   Column                          Dtype         
---  ------                          -----         
 0   CASE_NUMBER                     object        
 1   CASE_STATUS                     object        
 2   RECEIVED_DATE                   datetime64[ns]
 3   DECISION_DATE                   datetime64[ns]
 4   ORIGINAL_CERT_DATE              datetime64[ns]
 5   VISA_CLASS                      object        
 6   JOB_TITLE                       object        
 7   SOC_CODE                        object        
 8   SOC_TITLE                       object        
 9   FULL_TIME_POSITION              object        
 10  BEGIN_DATE                      datetime64[ns]
 11  END_DATE                        datetime64[ns]
 12  TOTAL_WORKER_POSITIONS          int64         
 13  NEW_EMPLOYMENT                  int64         
 14  CONTINUED_EMPLOYMENT            int64         
 1

In [102]:
import psycopg, os

In [96]:
print('Connecting to the PostgreSQL database...')
conn = psycopg.connect(
    host="localhost",
    port='5432',
    dbname="5400_h1b",
    user="postgres",
    password="123")

# create a cursor
cur = conn.cursor()

Connecting to the PostgreSQL database...


#### Create Tables

In [97]:
case_table = '''
CREATE TABLE IF NOT EXISTS "case" (
    case_number VARCHAR PRIMARY KEY,
    case_status VARCHAR,
    received_date TIMESTAMP,
    decision_date TIMESTAMP,
    original_cert_date TIMESTAMP,
    visa_class VARCHAR
);
'''

job_table = '''
CREATE TABLE IF NOT EXISTS job (
    case_number VARCHAR,
    job_title VARCHAR,
    soc_code VARCHAR,
    soc_title VARCHAR,
    full_time_position VARCHAR,
    begin_date TIMESTAMP,
    end_date TIMESTAMP,
    total_worker_positions INTEGER,
    new_employment INTEGER,
    continued_employment INTEGER,
    change_previous_employment INTEGER,
    new_concurrent_employment INTEGER,
    change_employer INTEGER,
    amended_petition INTEGER,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''
employer_table = '''
CREATE TABLE IF NOT EXISTS employer (
    case_number VARCHAR,
    employer_name VARCHAR,
    trade_name_dba VARCHAR,
    employer_address1 VARCHAR,
    employer_address2 VARCHAR,
    employer_city VARCHAR,
    employer_state VARCHAR,
    employer_postal_code VARCHAR,
    employer_country VARCHAR,
    employer_province VARCHAR,
    employer_phone VARCHAR,
    employer_phone_ext VARCHAR,
    naics_code INTEGER,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''

agent_attorney_table = '''
CREATE TABLE IF NOT EXISTS agent_attorney (
    case_number VARCHAR,
    agent_representing_employer VARCHAR,
    agent_attorney_last_name VARCHAR,
    agent_attorney_first_name VARCHAR,
    agent_attorney_middle_name VARCHAR,
    agent_attorney_address1 VARCHAR,
    agent_attorney_address2 VARCHAR,
    agent_attorney_city VARCHAR,
    agent_attorney_state VARCHAR,
    agent_attorney_postal_code VARCHAR,
    agent_attorney_country VARCHAR,
    agent_attorney_province VARCHAR,
    agent_attorney_phone VARCHAR,
    agent_attorney_phone_ext VARCHAR,
    agent_attorney_email_address VARCHAR,
    lawfirm_name_business_name VARCHAR,
    state_of_highest_court VARCHAR,
    name_of_highest_state_court VARCHAR,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''

worksite_table = '''
CREATE TABLE IF NOT EXISTS worksite (
    case_number VARCHAR,
    worksite_workers FLOAT,
    secondary_entity VARCHAR,
    secondary_entity_business_name VARCHAR,
    worksite_address1 VARCHAR,
    worksite_address2 VARCHAR,
    worksite_city VARCHAR,
    worksite_county VARCHAR,
    worksite_state VARCHAR,
    worksite_postal_code VARCHAR,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''

wage_table = '''
CREATE TABLE IF NOT EXISTS wage (
    case_number VARCHAR,
    wage_rate_of_pay_from FLOAT,
    wage_rate_of_pay_to FLOAT,
    wage_unit_of_pay VARCHAR,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''

prevailing_wage_table = '''
CREATE TABLE IF NOT EXISTS prevailing_wage (
    case_number VARCHAR,
    prevailing_wage FLOAT,
    pw_unit_of_pay VARCHAR,
    pw_tracking_number VARCHAR,
    pw_wage_level VARCHAR,
    pw_oes_year VARCHAR,
    pw_other_source VARCHAR,
    pw_other_year FLOAT,
    pw_survey_publisher VARCHAR,
    pw_survey_name VARCHAR,
    total_worksite_locations FLOAT,
    agree_to_lc_statement VARCHAR,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''

preparer_table = '''
CREATE TABLE IF NOT EXISTS preparer (
    case_number VARCHAR,
    preparer_last_name VARCHAR,
    preparer_first_name VARCHAR,
    preparer_middle_initial VARCHAR,
    preparer_business_name VARCHAR,
    preparer_email VARCHAR,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''

others_table = '''
CREATE TABLE IF NOT EXISTS others (
    case_number VARCHAR,
    h_1b_dependent VARCHAR,
    willful_violator VARCHAR,
    support_h1b VARCHAR,
    statutory_basis VARCHAR,
    appendix_a_attached VARCHAR,
    public_disclosure VARCHAR,
    FOREIGN KEY (case_number) REFERENCES "case" (case_number)
);
'''
cur.execute(case_table)
cur.execute(job_table)
cur.execute(employer_table)
cur.execute(agent_attorney_table)
cur.execute(worksite_table)
cur.execute(wage_table)
cur.execute(prevailing_wage_table)
cur.execute(preparer_table)
cur.execute(others_table)
conn.commit()

### Insert into tables

In [98]:
for index, row in combined_df.iterrows():
    # Insert data into case table
    cur.execute("""
    INSERT INTO "case" (case_number, case_status, received_date, decision_date, original_cert_date, visa_class)
    VALUES (%s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['CASE_STATUS'], row['RECEIVED_DATE'], row['DECISION_DATE'], row['ORIGINAL_CERT_DATE'], row['VISA_CLASS']))

    # Insert data into job table
    cur.execute("""
    INSERT INTO job (case_number, job_title, soc_code, soc_title, full_time_position, begin_date, end_date, total_worker_positions, new_employment, continued_employment, change_previous_employment, new_concurrent_employment, change_employer, amended_petition)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['JOB_TITLE'], row['SOC_CODE'], row['SOC_TITLE'], row['FULL_TIME_POSITION'], row['BEGIN_DATE'], row['END_DATE'], row['TOTAL_WORKER_POSITIONS'], row['NEW_EMPLOYMENT'], row['CONTINUED_EMPLOYMENT'], row['CHANGE_PREVIOUS_EMPLOYMENT'], row['NEW_CONCURRENT_EMPLOYMENT'], row['CHANGE_EMPLOYER'], row['AMENDED_PETITION']))

    # Insert data into employer table
    cur.execute("""
    INSERT INTO employer (case_number, employer_name, trade_name_dba, employer_address1, employer_address2, employer_city, employer_state, employer_postal_code, employer_country, employer_province, employer_phone, employer_phone_ext, naics_code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['EMPLOYER_NAME'], row['TRADE_NAME_DBA'], row['EMPLOYER_ADDRESS1'], row['EMPLOYER_ADDRESS2'], row['EMPLOYER_CITY'], row['EMPLOYER_STATE'], row['EMPLOYER_POSTAL_CODE'], row['EMPLOYER_COUNTRY'], row['EMPLOYER_PROVINCE'], row['EMPLOYER_PHONE'], row['EMPLOYER_PHONE_EXT'], row['NAICS_CODE']))

    # Insert data into agent_attorney table
    cur.execute("""
    INSERT INTO agent_attorney (case_number, agent_representing_employer, agent_attorney_last_name, agent_attorney_first_name, agent_attorney_middle_name, agent_attorney_address1, agent_attorney_address2, agent_attorney_city, agent_attorney_state, agent_attorney_postal_code, agent_attorney_country, agent_attorney_province, agent_attorney_phone, agent_attorney_phone_ext, agent_attorney_email_address, lawfirm_name_business_name, state_of_highest_court, name_of_highest_state_court)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['AGENT_REPRESENTING_EMPLOYER'], row['AGENT_ATTORNEY_LAST_NAME'], row['AGENT_ATTORNEY_FIRST_NAME'], row['AGENT_ATTORNEY_MIDDLE_NAME'], row['AGENT_ATTORNEY_ADDRESS1'], row['AGENT_ATTORNEY_ADDRESS2'], row['AGENT_ATTORNEY_CITY'], row['AGENT_ATTORNEY_STATE'], row['AGENT_ATTORNEY_POSTAL_CODE'], row['AGENT_ATTORNEY_COUNTRY'], row['AGENT_ATTORNEY_PROVINCE'], row['AGENT_ATTORNEY_PHONE'], row['AGENT_ATTORNEY_PHONE_EXT'], row['AGENT_ATTORNEY_EMAIL_ADDRESS'], row['LAWFIRM_NAME_BUSINESS_NAME'], row['STATE_OF_HIGHEST_COURT'], row['NAME_OF_HIGHEST_STATE_COURT']))
    
    # Insert data into worksite table
    cur.execute("""
    INSERT INTO worksite (case_number, worksite_workers, secondary_entity, secondary_entity_business_name, worksite_address1, worksite_address2, worksite_city, worksite_county, worksite_state, worksite_postal_code)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['WORKSITE_WORKERS'], row['SECONDARY_ENTITY'], row['SECONDARY_ENTITY_BUSINESS_NAME'], row['WORKSITE_ADDRESS1'], row['WORKSITE_ADDRESS2'], row['WORKSITE_CITY'], row['WORKSITE_COUNTY'], row['WORKSITE_STATE'], row['WORKSITE_POSTAL_CODE']))

    # Insert data into wage table
    cur.execute("""
    INSERT INTO wage (case_number, wage_rate_of_pay_from, wage_rate_of_pay_to, wage_unit_of_pay)
    VALUES (%s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['WAGE_RATE_OF_PAY_FROM'], row['WAGE_RATE_OF_PAY_TO'], row['WAGE_UNIT_OF_PAY']))

    # Insert data into prevailing_wage table
    cur.execute("""
    INSERT INTO prevailing_wage (case_number, prevailing_wage, pw_unit_of_pay, pw_tracking_number, pw_wage_level, pw_oes_year, pw_other_source, pw_other_year, pw_survey_publisher, pw_survey_name, total_worksite_locations, agree_to_lc_statement)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['PREVAILING_WAGE'], row['PW_UNIT_OF_PAY'], row['PW_TRACKING_NUMBER'], row['PW_WAGE_LEVEL'], row['PW_OES_YEAR'], row['PW_OTHER_SOURCE'], row['PW_OTHER_YEAR'], row['PW_SURVEY_PUBLISHER'], row['PW_SURVEY_NAME'], row['TOTAL_WORKSITE_LOCATIONS'], row['AGREE_TO_LC_STATEMENT']))

    # Insert data into preparer table
    cur.execute("""
    INSERT INTO preparer (case_number, preparer_last_name, preparer_first_name, preparer_middle_initial, preparer_business_name, preparer_email)
    VALUES (%s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['PREPARER_LAST_NAME'], row['PREPARER_FIRST_NAME'], row['PREPARER_MIDDLE_INITIAL'], row['PREPARER_BUSINESS_NAME'], row['PREPARER_EMAIL']))

    # Insert data into others table
    cur.execute("""
    INSERT INTO others (case_number, h_1b_dependent, willful_violator, support_h1b, statutory_basis, appendix_a_attached, public_disclosure)
    VALUES (%s, %s, %s, %s, %s, %s, %s);
    """, (row['CASE_NUMBER'], row['H_1B_DEPENDENT'], row['WILLFUL_VIOLATOR'], row['SUPPORT_H1B'], row['STATUTORY_BASIS'], row['APPENDIX_A_ATTACHED'], row['PUBLIC_DISCLOSURE']))
    

conn.commit()

In [100]:
cur.close()
conn.close()